In [110]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
import os
from sklearn.feature_selection import VarianceThreshold

import torch
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from keras.optimizers import SGD, Adam
from keras.regularizers import l2
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import f_regression, SelectKBest

from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression


import numpy as np

In [111]:
# Load the training data
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

used = []

# Extract features and target variable
X = train_data.drop('price_doc', axis=1)
y = train_data['price_doc']
X_test = test_data.drop(['row ID'], axis=1)

In [ ]:
# X = X.drop('sub_area', axis=1)
# X_test = X_test.drop('sub_area', axis=1)
# used.append('Removed sub_area')

In [112]:
X = pd.get_dummies(X)
X_test = pd.get_dummies(X_test) 
used.append('OneHot Encoding')

In [113]:
# Keep only the columns in X_test that are present in X
common_columns = X_test.columns.intersection(X.columns)
X = X[common_columns]


In [ ]:
X.shape

In [ ]:
X_test.shape

In [ ]:
from sklearn.calibration import LabelEncoder

categorical_columns = X.select_dtypes(include=['object']).columns.tolist()
print("Train: Categorical columns:", categorical_columns)

label_encoder = LabelEncoder()

for column in categorical_columns:
    X[column] = label_encoder.fit_transform(X[column])

categorical_columns_test = X_test.select_dtypes(include=['object']).columns.tolist()
print("Test: Categorical columns:", categorical_columns_test)

label_encoder = LabelEncoder()

for column in categorical_columns_test:
    X_test[column] = label_encoder.fit_transform(X_test[column])

used.append('Label Encoding')

In [ ]:
# # drop all columns in X_train with dtypes object
# for col in X.columns:
#     if X[col].dtype == 'object':
#         X.drop(col, axis=1, inplace=True)

# # drop all columns in X_test with dtypes object
# for col in X_test.columns:
#     if X_test[col].dtype == 'object':
#         X_test.drop(col, axis=1, inplace=True)

# used.append("Removed Object Dtypes")

In [114]:
X = X.astype('float32')
X_test = X_test.astype('float32')
used.append("Converted All Columns To float32")

In [ ]:
X.shape

In [116]:
from sklearn.decomposition import PCA
n_components =  100 # You can choose the number of components based on your requirements
pca = PCA(n_components=n_components)

# Apply PCA to the training data
X_pca = pca.fit_transform(X)

# Apply the same PCA transformation to the test data
df2_onehot_pca = pca.transform(X_test)

In [ ]:

#X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

variance_threshold = 0.01 # Set your desired threshold
selector = VarianceThreshold(threshold=variance_threshold)
X_var = selector.fit_transform(X)
X_test = selector.transform(X_test)
used.append('variance based filter')

In [ ]:
X_var.shape 

In [ ]:

# Feature selection using f_regression
selector = SelectKBest(score_func=f_regression, k='all')
X_selected = selector.fit_transform(X, y)

# Get feature scores
feature_scores = selector.scores_

# Print the feature scores
print("Feature Scores:")
for feature, score in zip(X.columns, feature_scores):
    print(f"{feature}: {score}")

# Select the top k features based on scores
k = 270
selected_features = X.columns[np.argsort(feature_scores)[::-1][:k]]

# Use only the top k features
X_selected = X[selected_features]

In [ ]:
ols_reg = LinearRegression()
sfs = SequentialFeatureSelector(ols_reg, direction='forward', n_features_to_select=50, n_jobs=-1)
sfs.fit(X, y)

# Get the selected feature indices
selected_feature_indices = sfs.get_support(indices=True)

# Print the names of the selected features
selected_features = X.columns[selected_feature_indices]
print("Selected Features:", selected_features)

In [ ]:
import statsmodels.api as sm
import pandas as pd

# Convert scaled_df1 to a DataFrame without specifying columns

# Add a constant term to the feature matrix
X_with_const = sm.add_constant(X)

# Fit a linear regression model
model = sm.OLS(y, X_with_const).fit()

# Get p-values for each feature
p_values = model.pvalues[1:]  # Exclude the constant term

# Set your desired threshold for p-value
threshold = 0.05

# Filter features based on p-value
selected_features = p_values[p_values < threshold].index

# Display selected features
print("Selected Features:")
print(selected_features)

# Select columns in the DataFrame
X_selected = X[selected_features]


In [ ]:
X_test = X_test[X_selected.columns]

In [ ]:
X_selected.shape

In [ ]:
X_test = X_test[X.columns]

In [ ]:
X_test = X_test[X.columns]


In [117]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_pca, y, test_size=0.3, random_state=42)

In [ ]:
#X_train.shape
X_val.shape

In [ ]:
#convert X_train to df
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

In [ ]:
# Keep only the columns in X_test that are present in X
common_columns = X_train.columns.intersection(X_test.columns)
X_test = X_test[common_columns]

# Convert to NumPy arrays for StandardScaler
X_test = X_test.values


In [ ]:
X_train.shape

In [118]:
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(df2_onehot_pca)
used.append("StandardScaler")

In [ ]:
X_train.shape

<h1>With Keras<h1>

In [ ]:
features_used = X_train.shape[1]

#optimizerUsing = SGD(learning_rate=0.001, momentum=0.9)

# optimizerUsing = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False)
optimizerUsing = Adam(lr=0.001)#, epsilon=1e-07)

# Build the neural network
os.environ["KERAS_BACKEND"] = "torch"
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))  # You can use either dropout or early stopping
model.add(Dense(100, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))
model.add(Dense(68, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))
model.add(Dense(40, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))
model.add(Dense(20, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))
model.add(Dense(1, activation='linear'))  # Output layer with linear activation for regression

# Compile the model
model.compile(loss='mean_squared_error', optimizer=optimizerUsing)

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=1800, validation_data=(X_val, y_val), callbacks=[early_stopping])

used.append("Keras With Torch Backend")
# used.append("1 Hidden Layer: Hidden Layer 1 with 140 neurons relu activation, Output linear")
# used.append("2 Hidden Layers: Hidden Layer 1 with 100 neurons relu activation, Hidden Layer 2 with 50 neurons relu activation, Dropout 0.3, Output linear")
used.append(" Hidden Layers: Hidden Layer 1 with 128 neurons relu activation L2 regularization, Hidden Layer 2 with 100 neurons relu activation L2 regularization,Hidden Layer 3 with 68 neurons relu activation L2 regularization Hidden Layer 4 with 40 neurons relu activation L2 regularization, Hidden Layer 5 with 20 neurons relu activation L2 regularization")
used.append("Output linear")
used.append("Optimizer: Adam(lr=0.01)")
#used.append("Optimizer: SGD(learning_rate=0.001, momentum=0.9)")
used.append("Loss Calculation: Mean Squared Error")
used.append("EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)")
used.append("batch_size=1800, epochs=100, early_stopping, dropout=0.3 (used in hidden layers only)")
# used.append("batch_size=1800, epochs=100, early_stopping")

# Make predictions
predictions = model.predict(X_test)

<h1>With Skilearn<h1>

c:\Users\anfam\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [120]:
# Create a DataFrame with the results
submission_df = pd.DataFrame({'row ID': test_data['row ID'], 'price_doc': predictions.flatten()})

# Save the results to a CSV file
submission_df.to_csv('predictions69.csv', index=False)

In [121]:
print("Features Used = " + str(features_used) + "\n")
for i in used:
    print(i)

Features Used = 100

OneHot Encoding
Converted All Columns To float32
log Normalization
StandardScaler
MLPRegressor
 Hidden Layer 1 with 128 neurons relu activation L2 regularization, Hidden Layer 2 with 100 neurons relu activation L2 regularization,Hidden Layer 3 with 68 neurons relu activation L2 regularization Hidden Layer 4 with 40 neurons relu activation L2 regularization, Hidden Layer 5 with 20 neurons relu activation L2 regularization, Output linear
Adam Optimizer
EarlyStopping
batch_size=1800, epochs=100, early_stopping, dropout=0.3 (used in hidden layers only)
